In [20]:
import keras as k
import scipy

In [13]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import model_from_json

from keras.models import Sequential
from keras.layers import Dense, Activation
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

dumpfiles(home=home, fsize=9, phi=0.1, dp = 75, timestep=0.6)

dumpfiles(home=home, fsize=9, phi=0.1, dp = 75, timestep=0.65)

dumpfiles(home=home, fsize=9, phi=0.1, dp = 150, timestep=0.65)

dumpfiles(home=home, fsize=9, phi=0.1, dp = 300, timestep=0.65)

dumpfiles(home=home, fsize=9, phi=0.3, dp = 75, timestep=0.425)

In [ ]:
home = '/home/nandhini/aozel/periodicBox'
fsize = 9
phi=0.1
dp = 75

data_signature1 = 'fsize' + str(fsize)+'phi'+str(phi)+'dp'+str(dp)+'timestep'+str(0.6)
data_signature2 = 'fsize' + str(fsize)+'phi'+str(phi)+'dp'+str(dp)+'timestep'+str(0.7)
data_signature3 = 'fsize' + str(9)+'phi'+str(0.3)+'dp'+str(75)+'timestep'+str(0.4)
data_signature4 = 'fsize' + str(9)+'phi'+str(0.1)+'dp'+str(300)+'timestep'+str(0.65)
data_signature5 = 'fsize' + str(9)+'phi'+str(0.1)+'dp'+str(150)+'timestep'+str(0.65)
data_signature6 = 'fsize' + str(9)+'phi'+str(0.3)+'dp'+str(75)+'timestep'+str(0.3) + 'HighCoh'




file_myData1 = "myDataUslip"+data_signature1+".pickle"
file_myData2 = "myDataUslip"+data_signature2+".pickle"
file_myData3 = "myDataUslip"+data_signature3+".pickle"
file_myData4 = "myDataUslip"+data_signature4+".pickle"
file_myData5 = "myDataUslip"+data_signature5+".pickle"
file_myData6 = "myDataUslip"+data_signature6+".pickle"





file_target1 = "targetUslip"+data_signature1+".pickle"
file_target2 = "targetUslip"+data_signature2+".pickle"
file_target3 = "targetUslip"+data_signature3+".pickle"
file_target4 = "targetUslip"+data_signature4+".pickle"
file_target5 = "targetUslip"+data_signature5+".pickle"
file_target6 = "targetUslip"+data_signature6+".pickle"




myData1 = pickle.load( open( file_myData1, "rb" ) )
myData2 = pickle.load( open( file_myData2, "rb" ) )
myData3 = pickle.load( open( file_myData3, "rb" ) )
myData4 = pickle.load( open( file_myData4, "rb" ) )
myData5 = pickle.load( open( file_myData5, "rb" ) )
myData6 = pickle.load( open( file_myData6, "rb" ) )


myData = np.vstack((myData1, myData2, myData3, myData4, myData5, myData6))
                

target1 = pickle.load( open( file_target1, "rb" ) )
target2 = pickle.load( open( file_target2, "rb" ) )
target3 = pickle.load( open( file_target3, "rb" ) )
target4 = pickle.load( open( file_target4, "rb" ) )
target5 = pickle.load( open( file_target5, "rb" ) )
target6 = pickle.load( open( file_target6, "rb" ) )


target = np.vstack((target1.reshape(-1,1),target2.reshape(-1,1),
                    target3.reshape(-1,1),target4.reshape(-1,1),target5.reshape(-1,1),
                    target6.reshape(-1,1)
                   ))

In [4]:
myData.shape, target.shape

((11608272, 108), (11608272, 1))

In [5]:
X_train, X_test, y_train, y_test = train_test_split(myData, target, test_size=0.02, random_state=42)
X_realCV, X_realTT, y_realCV, y_realTT = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [6]:
scaler = StandardScaler()
scaler.fit(X_train)
pickle.dump( scaler, open( 'scale'+data_signature1, "wb" ) )
X_train_norm = scaler.transform(X_train)
X_realCV_norm = scaler.transform(X_realCV)
X_realTT_norm = scaler.transform(X_realTT)

In [19]:
np.savetxt("drag_std.csv", scaler.std_, delimiter=",")
np.savetxt("drag_mean.csv", scaler.mean_, delimiter=",")

/home/hema/yile/anaconda3/lib/python3.5/site-packages/sklearn/utils/deprecation.py:70: DeprecationWarning: Function std_ is deprecated; Attribute ``std_`` will be removed in 0.19. Use ``scale_`` instead
  warnings.warn(msg, category=DeprecationWarning)


In [ ]:
X_train_norm.shape, X_realCV_norm.shape, X_realTT_norm.shape

((11376106, 108), (116083, 108), (116083, 108))

In [55]:
model = Sequential()
model.add(Dense(100, activation='relu', kernel_initializer='normal',input_dim=X_train_norm.shape[-1]))
model.add(Dense(100, activation='relu', kernel_initializer='normal'))
model.add(Dense(100, activation='relu', kernel_initializer='normal'))
model.add(Dense(100, activation='relu', kernel_initializer='normal'))
model.add(Dense(100, activation='relu', kernel_initializer='normal'))
model.add(Dense(100, activation='relu', kernel_initializer='normal'))
model.add(Dense(100, activation='relu', kernel_initializer='normal'))
model.add(Dense(64, activation='relu', kernel_initializer='normal'))
model.add(Dense(64, activation='relu', kernel_initializer='normal'))
model.add(Dense(64, activation='relu', kernel_initializer='normal'))
model.add(Dense(32, activation='relu', kernel_initializer='normal'))
model.add(Dense(16, activation='relu', kernel_initializer='normal'))
model.add(Dense(8, activation='relu', kernel_initializer='normal'))
model.add(Dense(4, activation='relu', kernel_initializer='normal'))
model.add(Dense(1, activation='linear', kernel_initializer='normal'))
model.compile(loss='mean_absolute_error', optimizer='Adam')
save_file = 'weights_Uslip3'+ data_signature1 + '.hdf5'
model.load_weights(save_file)
checkpointer = ModelCheckpoint(filepath=save_file, verbose=1, save_best_only=True)
#model.fit(X_train_norm, y_train, batch_size=1000, epochs=40, verbose=1, validation_data=(X_realCV_norm, y_realCV), callbacks=[checkpointer])

In [56]:
# serialize model to JSON
model_json = model.to_json()
with open("drag2.json", "w") as json_file:
    json_file.write(model_json)

In [15]:
data_signature1

'fsize9phi0.1dp75timestep0.6'

In [52]:
model.predict(np.array([[2.70287, 0.0280261, 1.23125, -0.769839, 2.70308, 0.0279703, 1.23075, -0.769849, 2.70327, 0.0278884, 1.23002, -0.769865, 2.70358, 0.0282558, 1.23087, -0.769851, 2.70378, 0.0281992, 1.23037, -0.769862, 2.70398, 0.0281165, 1.22965, -0.769879, 2.7043, 0.0284953, 1.23039, -0.769917, 2.70451, 0.0284387, 1.2299, -0.76993, 2.70471, 0.0283554, 1.22919, -0.769949, 2.70326, 0.0279363, 1.23085, -0.769817, 2.70346, 0.027879, 1.23035, -0.769827, 2.70366, 0.027796, 1.22961, -0.769845, 2.70396, 0.0281659, 1.23047, -0.769828, 2.70417, 0.0281084, 1.22998, -0.76984, 2.70437, 0.0280249, 1.22925, -0.769858, 2.70468, 0.0284036, 1.22999, -0.769895, 2.70489, 0.0283462, 1.22951, -0.769908, 2.70509, 0.0282624, 1.22879, -0.769928, 2.70362, 0.0278401, 1.2305, -0.769814, 2.70383, 0.0277819, 1.22999, -0.769825, 2.70403, 0.0276979, 1.22925, -0.769844, 2.70432, 0.028069, 1.23012, -0.769824, 2.70453, 0.0280113, 1.22962, -0.769836, 2.70473, 0.0279274, 1.22889, -0.769856, 2.70504, 0.0283043, 1.22965, -0.769891, 2.70525, 0.028247, 1.22915, -0.769905, 2.70545, 0.0281631, 1.22843, -0.769926]]))

array([[ 1.28914738]], dtype=float32)

In [53]:
a = np.array([[-1.16182, 0.0280261, 1.23125, -0.769839, -1.16181, 0.0279703, 1.23075, -0.769849, -1.16182, 0.0278884, 1.23002, -0.769865, -1.16182, 0.0282558, 1.23087, -0.769851, -1.16182, 0.0281992, 1.23037, -0.769862, -1.16183, 0.0281165, 1.22965, -0.769879, -1.16177, 0.0284953, 1.23039, -0.769917, -1.16177, 0.0284387, 1.2299, -0.76993, -1.16178, 0.0283554, 1.22919, -0.769949, -1.16171, 0.0279363, 1.23085, -0.769817, -1.16171, 0.027879, 1.23035, -0.769827, -1.16171, 0.027796, 1.22961, -0.769845, -1.16171, 0.0281659, 1.23047, -0.769828, -1.16171, 0.0281084, 1.22998, -0.76984, -1.16172, 0.0280249, 1.22925, -0.769858, -1.16166, 0.0284036, 1.22999, -0.769895, -1.16166, 0.0283462, 1.22951, -0.769908, -1.16167, 0.0282624, 1.22879, -0.769928, -1.16162, 0.0278401, 1.2305, -0.769814, -1.16161, 0.0277819, 1.22999, -0.769825, -1.16162, 0.0276979, 1.22925, -0.769844, -1.16161, 0.028069, 1.23012, -0.769824, -1.16161, 0.0280113, 1.22962, -0.769836, -1.16162, 0.0279274, 1.22889, -0.769856, -1.16156, 0.0283043, 1.22965, -0.769891, -1.16156, 0.028247, 1.22915, -0.769905, -1.16157, 0.0281631, 1.22843, -0.769926]])

In [54]:
model.predict(a)

array([[-0.33250952]], dtype=float32)

In [ ]:
model = Sequential()

model.add(Dense(100, activation='relu', kernel_initializer='normal',input_dim=X_train_norm.shape[-1]))
model.add(Dense(100, activation='relu', kernel_initializer='normal'))
model.add(Dense(100, activation='relu', kernel_initializer='normal'))
model.add(Dense(100, activation='relu', kernel_initializer='normal'))
model.add(Dense(100, activation='relu', kernel_initializer='normal'))
model.add(Dense(100, activation='relu', kernel_initializer='normal'))
model.add(Dense(100, activation='relu', kernel_initializer='normal'))
model.add(Dense(64, activation='relu', kernel_initializer='normal'))
model.add(Dense(64, activation='relu', kernel_initializer='normal'))
model.add(Dense(64, activation='relu', kernel_initializer='normal'))
model.add(Dense(32, activation='relu', kernel_initializer='normal'))
model.add(Dense(16, activation='relu', kernel_initializer='normal'))
model.add(Dense(8, activation='relu', kernel_initializer='normal'))
model.add(Dense(4, activation='relu', kernel_initializer='normal'))
model.add(Dense(1, activation='linear', kernel_initializer='normal'))
model.compile(loss='mean_absolute_error', optimizer='Adam')
save_file = 'weights_Uslip3'+ data_signature1 + '.hdf5'
model.load_weights(save_file)
checkpointer = ModelCheckpoint(filepath=save_file, verbose=1, save_best_only=True)
#model.fit(X_train_norm, y_train, batch_size=1000, epochs=40, verbose=1, validation_data=(X_realCV_norm, y_realCV), callbacks=[checkpointer])

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("drag2.json", "w") as json_file:
    json_file.write(model_json)

In [57]:
model = Sequential()
model.add(Dense(100,input_dim=108))
model.add(Activation('relu'))
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dense(100))
model.add(Activation('relu'))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(32))
model.add(Activation('relu'))

model.add(Dense(16))
model.add(Activation('relu'))

model.add(Dense(8))
model.add(Activation('relu'))

model.add(Dense(4))
model.add(Activation('relu'))

model.add(Dense(1))
model.compile(loss='mean_absolute_error', optimizer='Adam')
save_file = 'weights_Uslip3'+ data_signature1 + '.hdf5'
model.load_weights(save_file)

In [58]:
# serialize model to JSON
model_json = model.to_json()
with open("drag_correct.json", "w") as json_file:
    json_file.write(model_json)